In [37]:
import numpy as np
import numpy.typing as npt
import cProfile

# create a seed
seed = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=np.int8)

# create a biome
biome = np.zeros((100, 100), dtype=np.int8)
biome[0:10, 0:10] = seed

class Biome():
    def __init__(self, grid: npt.NDArray[np.int8]):
        self.grid = grid

    def sense_quarum(self, x: int, y: int) -> int:
        return sum([self.grid[x-1, y-1], self.grid[x, y-1], self.grid[x+1, y-1],
                    self.grid[x-1, y], self.grid[x+1, y],
                    self.grid[x-1, y+1], self.grid[x, y+1], self.grid[x+1, y+1]])

    def get_alive_cells(self) -> npt.NDArray[np.int8]:
        return np.array(np.where(self.grid == 1)).T

    def get_neighbor_indices(self) -> npt.NDArray[np.int8]:
        alive_cells = self.get_alive_cells()
        neighbor_indices = np.array(
            [
            [alive_cells[:, 0]-1, alive_cells[:, 1]-1],
            [alive_cells[:, 0], alive_cells[:, 1]-1],
            [alive_cells[:, 0]+1, alive_cells[:, 1]-1],
            [alive_cells[:, 0]-1, alive_cells[:, 1]],
            [alive_cells[:, 0]+1, alive_cells[:, 1]],
            [alive_cells[:, 0]-1, alive_cells[:, 1]+1],
            [alive_cells[:, 0], alive_cells[:, 1]+1],
            [alive_cells[:, 0]+1, alive_cells[:, 1]+1]
            ]
        ).T
        return neighbor_indices

    def apply_rules(self) -> npt.NDArray[np.int8]:
        neighbor_indices = self.get_neighbor_indices()
        neighbor_indices = neighbor_indices.reshape(-1, 2)
        neighbor_indices = np.unique(neighbor_indices, axis=0)
        neighbor_indices = neighbor_indices[
            (neighbor_indices[:, 0] >= 0) &
            (neighbor_indices[:, 0] < self.grid.shape[0]) &
            (neighbor_indices[:, 1] >= 0) &
            (neighbor_indices[:, 1] < self.grid.shape[1])
        ]
        neighbor_counts = np.zeros(self.grid.shape, dtype=np.int8)
        for i in range(neighbor_indices.shape[0]):
            neighbor_counts[neighbor_indices[i, 0], neighbor_indices[i, 1]] = \
                self.sense_quarum(neighbor_indices[i, 0], neighbor_indices[i, 1])
        return np.where(
            (neighbor_counts == 3) |
            (neighbor_counts == 2) & (self.grid == 1),
            1, 0
        )
    
    def run(self, n: int) -> None:
        for i in range(n):
            self.grid = self.apply_rules()



In [43]:
biome = Biome(biome)



IndexError: index 1 is out of bounds for axis 1 with size 1